In [12]:
import os
import sys
import pandas as pd

TOP_DIR=os.path.abspath("../../")
if not TOP_DIR in sys.path: sys.path.append(TOP_DIR)

In [13]:
from prepare import LMS_EXTRACT, DASHBOARD_DIR, create_table, save_files, summarise
from scripts.util.metadata import read_meta, filter_for_dataset, extract_dates

Load in the LMS data

In [14]:
lms_extract = pd.read_csv(
    LMS_EXTRACT,
    parse_dates=['lms_period']
)

Extract relevant metrics, pivot and save file for labour market

In [15]:
labour_market_status = lms_extract.pipe(create_table, [
    'JN5R', 'MGUQ', 'MGVF', 'MGVU', 'AIVZ', 'MGWY', 'AIYL',
    'JN6B', 'AGNJ', 'AGOL', 'AGPM', 'AIWI', 'AIXT', 'AIYU',
    'JN62', 'AGNT', 'AGOU', 'AGPV', 'AIXB', 'AIYC', 'AIZD',
    'JN69', 'AGNH', 'AGOJ', 'AGPK', 'AIWG', 'AIXR', 'AIYS',
    'JN6A', 'AGNI', 'AGOK', 'AGPL', 'AIWH', 'AIXS', 'AIYT',
    'JN6E', 'AGNM', 'AGOO', 'AGPP', 'AIWL', 'AIXW', 'AIYX',
    'JN6H', 'AGNP', 'AGOR', 'AGPS', 'AIWX', 'AIXZ', 'AIZA',
]).pipe(save_files, 'labour_market_status')
labour_market_status

variable,age_16_to_17_not_in_ft_education_in_employment_sa,age_18_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_not_in_ft_education_in_employment_sa,men_16_to_24_not_in_ft_education_in_employment_sa,women_16_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_in_ft_education_in_employment_sa,age_16_to_17_not_in_ft_education_unemployed_sa,age_18_to_24_not_in_ft_education_unemployed_sa,age_16_to_24_not_in_ft_education_unemployed_sa,men_16_to_24_not_in_ft_education_unemployed_sa,...,age_18_to_24_not_in_ft_education_total_sa,age_16_to_24_not_in_ft_education_total_sa,men_16_to_24_not_in_ft_education_total_sa,women_16_to_24_not_in_ft_education_total_sa,age_16_to_24_in_employment_sa,age_16_to_24_unemployed_sa,age_16_to_24_economically_inactive_sa,age_16_to_24_unemployment_rate_sa,quarter_label,quarter_axis_label
lms_period,,,,,,,,,,,,,,,,,,,,,
1992-04-01,314.0,3482.0,3796.0,2007.0,1789.0,685.0,100.0,674.0,774.0,505.0,...,4799.0,5287.0,2670.0,2618.0,4481.0,860.0,1820.0,16.1,Mar-May 1992,Mar-May\n1992
1992-07-01,301.0,3374.0,3675.0,1957.0,1719.0,700.0,103.0,673.0,776.0,503.0,...,4698.0,5163.0,2610.0,2553.0,4375.0,870.0,1853.0,16.6,Jun-Aug 1992,Jun-Aug\n1992
1992-10-01,295.0,3335.0,3630.0,1926.0,1704.0,675.0,94.0,706.0,800.0,515.0,...,4640.0,5092.0,2570.0,2521.0,4305.0,888.0,1842.0,17.1,Sep-Nov 1992,Sep-Nov\n1992
1993-01-01,280.0,3293.0,3573.0,1886.0,1687.0,644.0,89.0,726.0,815.0,523.0,...,4623.0,5059.0,2555.0,2504.0,4216.0,923.0,1833.0,18.0,Dec-Feb 1993,Dec-Feb\n1993
1993-04-01,270.0,3247.0,3517.0,1865.0,1652.0,629.0,86.0,723.0,810.0,521.0,...,4607.0,5044.0,2549.0,2495.0,4146.0,907.0,1856.0,18.0,Mar-May 1993,Mar-May\n1993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-01,76.0,2741.0,2817.0,1463.0,1354.0,925.0,25.0,255.0,281.0,182.0,...,3541.0,3722.0,1948.0,1774.0,3742.0,434.0,2645.0,10.4,Mar-May 2022,Mar-May\n2022
2022-07-01,74.0,2716.0,2789.0,1448.0,1342.0,952.0,22.0,203.0,225.0,143.0,...,3442.0,3610.0,1874.0,1736.0,3741.0,372.0,2724.0,9.0,Jun-Aug 2022,Jun-Aug\n2022
2022-10-01,66.0,2661.0,2727.0,1397.0,1330.0,965.0,22.0,282.0,303.0,191.0,...,3477.0,3660.0,1897.0,1763.0,3692.0,461.0,2686.0,11.1,Sep-Nov 2022,Sep-Nov\n2022


Extract relevant metrics for long-term unemployment, pivot and save file

In [16]:
long_term_unemployed = lms_extract.pipe(create_table, [
    'YBVH', 'YBXG', 'YBXJ', 'YBXM',
    'YBVN', 'YBXV', 'YBXY', 'YBYB',
])
long_term_unemployed = pd.DataFrame({
    'age_16_to_24_unemployed_sa': (long_term_unemployed.age_18_to_24_unemployed_sa + long_term_unemployed.age_16_to_17_unemployed_sa).round(0),
    'age_16_to_24_unemployed_6_to_12_months_sa':  (long_term_unemployed.age_18_to_24_unemployed_6_to_12_months_sa + long_term_unemployed.age_16_to_17_unemployed_6_to_12_months_sa).round(0),
    'age_16_to_24_unemployed_over_12_months_sa': (long_term_unemployed.age_18_to_24_unemployed_over_12_months_sa + long_term_unemployed.age_16_to_17_unemployed_over_12_months_sa).round(0),
    'quarter_label': long_term_unemployed.quarter_label,
    'quarter_axis_label': long_term_unemployed.quarter_label.str.replace(' ', '\\n'),
})
long_term_unemployed['age_16_to_24_unemployed_6_to_12_months_rate_sa'] = (long_term_unemployed['age_16_to_24_unemployed_6_to_12_months_sa'] / \
    long_term_unemployed['age_16_to_24_unemployed_sa'] * 100).round(1)
long_term_unemployed['age_16_to_24_unemployed_over_12_months_rate_sa'] = (long_term_unemployed['age_16_to_24_unemployed_over_12_months_sa'] / \
    long_term_unemployed['age_16_to_24_unemployed_sa'] * 100).round(1)
long_term_unemployed.pipe(save_files, 'long_term_unemployed')

,age_16_to_24_unemployed_sa,age_16_to_24_unemployed_6_to_12_months_sa,age_16_to_24_unemployed_over_12_months_sa,quarter_label,quarter_axis_label,age_16_to_24_unemployed_6_to_12_months_rate_sa,age_16_to_24_unemployed_over_12_months_rate_sa
lms_period,,,,,,,
1992-04-01,860.0,177.0,209.0,Mar-May 1992,Mar-May\n1992,20.6,24.3
1992-07-01,869.0,184.0,239.0,Jun-Aug 1992,Jun-Aug\n1992,21.2,27.5
1992-10-01,888.0,186.0,243.0,Sep-Nov 1992,Sep-Nov\n1992,20.9,27.4
1993-01-01,923.0,189.0,269.0,Dec-Feb 1993,Dec-Feb\n1993,20.5,29.1
1993-04-01,907.0,181.0,280.0,Mar-May 1993,Mar-May\n1993,20.0,30.9
...,...,...,...,...,...,...,...
2022-04-01,434.0,36.0,64.0,Mar-May 2022,Mar-May\n2022,8.3,14.7
2022-07-01,372.0,43.0,61.0,Jun-Aug 2022,Jun-Aug\n2022,11.6,16.4
2022-10-01,461.0,60.0,NaN,Sep-Nov 2022,Sep-Nov\n2022,13.0,NaN


Create a summary file

In [17]:
summary = pd.concat([
    summarise(
      long_term_unemployed=long_term_unemployed,
      labour_market_status=labour_market_status,
    ),
    read_meta().pipe(filter_for_dataset, 'LMS').pipe(extract_dates),
])

summary.to_json(os.path.join(DASHBOARD_DIR, 'latest.json'), indent=2, date_format='iso')